# Notebook Setup and Load Models

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Imports

In [2]:
!pip install -q tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 41.6 MB/s eta 0:00:00


In [3]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 94.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 28.0 MB/s eta 0:00:00


In [4]:
!pip3 install -q emoji==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 201.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [5]:
!pip install -q tensorflow_text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 53.7 MB/s eta 0:00:00


In [6]:
!pip install -q gensim==3.8.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 69.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [7]:
!pip install -q pyyaml h5py  # Required to save models in HDF5 format

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_text as tf_text
from tensorflow import keras
import transformers
from transformers import AutoTokenizer,TFBertModel, TFRobertaModel

import nltk
from nltk.corpus import reuters
from nltk.data import find

import gensim

/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


## Functions and Constants

In [9]:
def to_arrays(df):
  X = df['comment_text'].to_numpy()
  y = df['toxicity_binary'].to_numpy()
  return X, y

In [10]:
MAX_SEQUENCE_LENGTH = 128

In [11]:
def tokenizing_pipeline(X, tokenizer):
  bert_tokenized = tokenizer(list(X),
                max_length=MAX_SEQUENCE_LENGTH,
                truncation=True,
                padding='max_length',
                return_tensors='tf')
  bert_inputs = [bert_tokenized.input_ids,
                 bert_tokenized.token_type_ids,
                 bert_tokenized.attention_mask]
  return bert_inputs

In [12]:
# BELOW IS FOR CNN INPUT SETUP
nltk.download('word2vec_sample')

word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

# EMBEDDING MATRIX AND VOCAB DICTIONARY
EMBEDDING_DIM = len(model['university'])      # we know... it's 300

# initialize embedding matrix and word-to-id map:
embedding_matrix = np.zeros((len(model.vocab.keys()) + 1, EMBEDDING_DIM))       
vocab_dict = {}

# build the embedding matrix and the word-to-id map:
for i, word in enumerate(model.vocab.keys()):
    embedding_vector = model[word]

    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        vocab_dict[word] = i

# we can use the last index at the end of the vocab for unknown tokens
vocab_dict['[UNK]'] = len(vocab_dict)

def docs_to_vocab_ids(tokenized_texts_list):
    """
    converting a list of strings to a list of lists of word ids
    """
    texts_vocab_ids = []
    for i, token_list in enumerate(tokenized_texts_list):

        # Get the vocab id for each token in this doc ([UNK] if not in vocab)
        vocab_ids = []
        for token in list(token_list.numpy()):
            decoded = token.decode('utf-8', errors='ignore')
            if decoded in vocab_dict:
                vocab_ids.append(vocab_dict[decoded])
            else:
                vocab_ids.append(vocab_dict['[UNK]'])
            
        # Truncate text to max length, add padding up to max length
        vocab_ids = vocab_ids[:MAX_SEQUENCE_LENGTH]
        n_padding = (MAX_SEQUENCE_LENGTH - len(vocab_ids))
        # For simplicity in this model, we'll just pad with uknown tokens
        vocab_ids += [vocab_dict['[UNK]']] * n_padding

        # Add this example to the list of converted docs
        texts_vocab_ids.append(vocab_ids)
            
        if i % 5000 == 0:
            print('Examples processed: ', i)
        
    print('Total examples: ', i)

    return np.array(texts_vocab_ids)

[nltk_data] Downloading package word2vec_sample to /root/nltk_data...
[nltk_data]   Unzipping models/word2vec_sample.zip.


## Load Evaluation Data

In [13]:
# read csv files
augmented_disability_df_test = pd.read_csv('drive/MyDrive/data/disability-dataset-test-augmented.csv')
augmented_race_df_test = pd.read_csv('drive/MyDrive/data/race-dataset-test-augmented.csv')
augmented_gender_df_test = pd.read_csv('drive/MyDrive/data/gender-dataset-test-augmented.csv')
augmented_nationality_df_test = pd.read_csv('drive/MyDrive/data/nationality-dataset-test-augmented.csv')
augmented_sexual_orientation_df_test = pd.read_csv('drive/MyDrive/data/sexual-orientation-dataset-test-augmented.csv')
augmented_religion_df_test = pd.read_csv('drive/MyDrive/data/religion-dataset-test-augmented.csv')

hatecheck_disability_df_test = pd.read_csv('drive/MyDrive/data/disability-dataset-hatecheck.csv')
hatecheck_race_df_test = pd.read_csv('drive/MyDrive/data/race-dataset-hatecheck.csv')
hatecheck_gender_df_test = pd.read_csv('drive/MyDrive/data/gender-dataset-hatecheck.csv')
hatecheck_nationality_df_test = pd.read_csv('drive/MyDrive/data/nationality-dataset-hatecheck.csv')
hatecheck_sexual_orientation_df_test = pd.read_csv('drive/MyDrive/data/sexual_orientation-dataset-hatecheck.csv')
hatecheck_religion_df_test = pd.read_csv('drive/MyDrive/data/religion-dataset-hatecheck.csv')

jigsaw_disability_df_test = pd.read_csv('drive/MyDrive/data/disability-dataset-jigsaw.csv')
jigsaw_race_df_test = pd.read_csv('drive/MyDrive/data/race-dataset-jigsaw.csv')
jigsaw_gender_df_test = pd.read_csv('drive/MyDrive/data/gender-dataset-jigsaw.csv')
# jigsaw_nationality_df_test = pd.read_csv('drive/MyDrive/data/nationality-dataset-jigsaw.csv') # jigsaw doesn't have nationality
jigsaw_sexual_orientation_df_test = pd.read_csv('drive/MyDrive/data/sexual_orientation-dataset-jigsaw.csv')
jigsaw_religion_df_test = pd.read_csv('drive/MyDrive/data/religion-dataset-jigsaw.csv')

In [14]:
def to_arrays(df):
  X = df['comment_text'].to_numpy()
  y = df['toxicity_binary'].to_numpy()
  return X, y

# convert to numpy arrays
X_test_augmented_disability, y_test_augmented_disability = to_arrays(augmented_disability_df_test)
X_test_augmented_race, y_test_augmented_race = to_arrays(augmented_race_df_test)
X_test_augmented_gender, y_test_augmented_gender = to_arrays(augmented_gender_df_test)
X_test_augmented_nationality, y_test_augmented_nationality = to_arrays(augmented_nationality_df_test)
X_test_augmented_sexual_orientation, y_test_augmented_sexual_orientation = to_arrays(augmented_sexual_orientation_df_test)
X_test_augmented_religion, y_test_augmented_religion = to_arrays(augmented_religion_df_test)

X_test_hatecheck_disability, y_test_hatecheck_disability = to_arrays(hatecheck_disability_df_test)
X_test_hatecheck_race, y_test_hatecheck_race = to_arrays(hatecheck_race_df_test)
X_test_hatecheck_gender, y_test_hatecheck_gender = to_arrays(hatecheck_gender_df_test)
X_test_hatecheck_nationality, y_test_hatecheck_nationality = to_arrays(hatecheck_nationality_df_test)
X_test_hatecheck_sexual_orientation, y_test_hatecheck_sexual_orientation = to_arrays(hatecheck_sexual_orientation_df_test)
X_test_hatecheck_religion, y_test_hatecheck_religion = to_arrays(hatecheck_religion_df_test)

X_test_jigsaw_disability, y_test_jigsaw_disability = to_arrays(jigsaw_disability_df_test)
X_test_jigsaw_race, y_test_jigsaw_race = to_arrays(jigsaw_race_df_test)
X_test_jigsaw_gender, y_test_jigsaw_gender = to_arrays(jigsaw_gender_df_test)
# X_test_jigsaw_nationality, y_test_jigsaw_nationality = to_arrays(jigsaw_nationality_df_test) # jigsaw doesn't have nationality
X_test_jigsaw_sexual_orientation, y_test_jigsaw_sexual_orientation = to_arrays(jigsaw_sexual_orientation_df_test)
X_test_jigsaw_religion, y_test_jigsaw_religion = to_arrays(jigsaw_religion_df_test)

# Convert to tensors for CNN
cnn_augmented_test_disability_labels = tf.convert_to_tensor(augmented_disability_df_test['toxicity_binary'])
cnn_hatecheck_test_disability_labels = tf.convert_to_tensor(hatecheck_disability_df_test['toxicity_binary'])
cnn_jigsaw_test_disability_labels = tf.convert_to_tensor(jigsaw_disability_df_test['toxicity_binary'])

cnn_augmented_test_disability_examples = tf.convert_to_tensor(augmented_disability_df_test['comment_text'])
cnn_hatecheck_test_disability_examples = tf.convert_to_tensor(hatecheck_disability_df_test['comment_text'])
cnn_jigsaw_test_disability_examples = tf.convert_to_tensor(jigsaw_disability_df_test['comment_text'])

Sanity check: Inspect some of the loaded data

In [15]:
X_test_augmented_disability[2]

" ADD/ADHD is a mental disorder, they don't exist! its just stupid behavior!\n- ADD is an excuse for bad parenting\n-"

In [16]:
X_test_hatecheck_race[10]

'These black people disgust me so much. '

In [17]:
X_test_jigsaw_religion[20]

'Layla: I think that you are looking at this backwards.  The question is not "how many Muslims are not affected by this?".  The question is "how many non-Muslims are affected by this?".  Since the answer to the second question is almost certainly none, we can conclude that this law targets Muslims wearing face-covering attire.'

## Load Base BERT Models and  Tokenizers

In [18]:
bertcased_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
bertcased_model = TFBertModel.from_pretrained("bert-base-cased")

bertweet_tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base",
                                                    use_fast=False,
                                                    normalization=True,
                                                    add_special_tokens=True,
                                                    return_attention_mask=True)
bertweet_model = TFRobertaModel.from_pretrained("vinai/bertweet-base")

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Some layers from the model checkpoint at vinai/bertweet-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at vinai/bertweet-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


## Tokenize Data for BERT Cased and BERTweet

In [19]:
# CNN
# tokenize
cnn_tokenizer = tf_text.WhitespaceTokenizer()
cnn_augmented_disability_test_tokens = cnn_tokenizer.tokenize(cnn_augmented_test_disability_examples)
cnn_hatecheck_disability_test_tokens = cnn_tokenizer.tokenize(cnn_hatecheck_test_disability_examples)
cnn_jigsaw_disability_test_tokens = cnn_tokenizer.tokenize(cnn_jigsaw_test_disability_examples)

# cnn inputs
cnn_augmented_disability_test_input = docs_to_vocab_ids(cnn_augmented_disability_test_tokens)
cnn_hatecheck_disability_test_input = docs_to_vocab_ids(cnn_hatecheck_disability_test_tokens)
cnn_jigsaw_disability_test_input = docs_to_vocab_ids(cnn_jigsaw_disability_test_tokens)

# labels
cnn_augmented_disability_test_labels = np.array(cnn_augmented_test_disability_labels)
cnn_hatecheck_disability_test_labels = np.array(cnn_hatecheck_test_disability_labels)
cnn_jigsaw_disability_test_labels = np.array(cnn_jigsaw_test_disability_labels)

Examples processed:  0
Examples processed:  5000
Total examples:  7264
Examples processed:  0
Total examples:  483
Examples processed:  0
Total examples:  1537


In [20]:
# BERT Cased
bertcased_augmented_test_inputs_disability = tokenizing_pipeline(X_test_augmented_disability, bertcased_tokenizer)
bertcased_augmented_test_inputs_race = tokenizing_pipeline(X_test_augmented_race, bertcased_tokenizer)
bertcased_augmented_test_inputs_gender = tokenizing_pipeline(X_test_augmented_gender, bertcased_tokenizer)
bertcased_augmented_test_inputs_nationality = tokenizing_pipeline(X_test_augmented_nationality, bertcased_tokenizer)
bertcased_augmented_test_inputs_sexual_orientation = tokenizing_pipeline(X_test_augmented_sexual_orientation, bertcased_tokenizer)
bertcased_augmented_test_inputs_religion = tokenizing_pipeline(X_test_augmented_religion, bertcased_tokenizer)

bertcased_hatecheck_test_inputs_disability = tokenizing_pipeline(X_test_hatecheck_disability, bertcased_tokenizer)
bertcased_hatecheck_test_inputs_race = tokenizing_pipeline(X_test_hatecheck_race, bertcased_tokenizer)
bertcased_hatecheck_test_inputs_gender = tokenizing_pipeline(X_test_hatecheck_gender, bertcased_tokenizer)
bertcased_hatecheck_test_inputs_nationality = tokenizing_pipeline(X_test_hatecheck_nationality, bertcased_tokenizer)
bertcased_hatecheck_test_inputs_sexual_orientation = tokenizing_pipeline(X_test_hatecheck_sexual_orientation, bertcased_tokenizer)
bertcased_hatecheck_test_inputs_religion = tokenizing_pipeline(X_test_hatecheck_religion, bertcased_tokenizer)

bertcased_jigsaw_test_inputs_disability = tokenizing_pipeline(X_test_jigsaw_disability, bertcased_tokenizer)
bertcased_jigsaw_test_inputs_race = tokenizing_pipeline(X_test_jigsaw_race, bertcased_tokenizer)
bertcased_jigsaw_test_inputs_gender = tokenizing_pipeline(X_test_jigsaw_gender, bertcased_tokenizer)
# bertcased_jigsaw_test_inputs_nationality = tokenizing_pipeline(X_test_jigsaw_nationality, bertcased_tokenizer) # jigsaw doesn't have nationality
bertcased_jigsaw_test_inputs_sexual_orientation = tokenizing_pipeline(X_test_jigsaw_sexual_orientation, bertcased_tokenizer)
bertcased_jigsaw_test_inputs_religion = tokenizing_pipeline(X_test_jigsaw_religion, bertcased_tokenizer)

In [21]:
# BERTweet
bertweet_augmented_test_inputs_disability = tokenizing_pipeline(X_test_augmented_disability, bertweet_tokenizer)
bertweet_augmented_test_inputs_race = tokenizing_pipeline(X_test_augmented_race, bertweet_tokenizer)
bertweet_augmented_test_inputs_gender = tokenizing_pipeline(X_test_augmented_gender, bertweet_tokenizer)
bertweet_augmented_test_inputs_nationality = tokenizing_pipeline(X_test_augmented_nationality, bertweet_tokenizer)
bertweet_augmented_test_inputs_sexual_orientation = tokenizing_pipeline(X_test_augmented_sexual_orientation, bertweet_tokenizer)
bertweet_augmented_test_inputs_religion = tokenizing_pipeline(X_test_augmented_religion, bertweet_tokenizer)

bertweet_hatecheck_test_inputs_disability = tokenizing_pipeline(X_test_hatecheck_disability, bertweet_tokenizer)
bertweet_hatecheck_test_inputs_race = tokenizing_pipeline(X_test_hatecheck_race, bertweet_tokenizer)
bertweet_hatecheck_test_inputs_gender = tokenizing_pipeline(X_test_hatecheck_gender, bertweet_tokenizer)
bertweet_hatecheck_test_inputs_nationality = tokenizing_pipeline(X_test_hatecheck_nationality, bertweet_tokenizer)
bertweet_hatecheck_test_inputs_sexual_orientation = tokenizing_pipeline(X_test_hatecheck_sexual_orientation, bertweet_tokenizer)
bertweet_hatecheck_test_inputs_religion = tokenizing_pipeline(X_test_hatecheck_religion, bertweet_tokenizer)

bertweet_jigsaw_test_inputs_disability = tokenizing_pipeline(X_test_jigsaw_disability, bertweet_tokenizer)
bertweet_jigsaw_test_inputs_race = tokenizing_pipeline(X_test_jigsaw_race, bertweet_tokenizer)
bertweet_jigsaw_test_inputs_gender = tokenizing_pipeline(X_test_jigsaw_gender, bertweet_tokenizer)
# bertweet_jigsaw_test_inputs_nationality = tokenizing_pipeline(X_test_jigsaw_nationality, bertweet_tokenizer) # jigsaw doesn't have nationality
bertweet_jigsaw_test_inputs_sexual_orientation = tokenizing_pipeline(X_test_jigsaw_sexual_orientation, bertweet_tokenizer)
bertweet_jigsaw_test_inputs_religion = tokenizing_pipeline(X_test_jigsaw_religion, bertweet_tokenizer)

## Define Build Functions for BERT

In [22]:
def build_bertcased_cls_model(max_sequence_length=MAX_SEQUENCE_LENGTH,
                          hidden_size=100, 
                          dropout=0.3,
                          learning_rate=0.0001,
                          num_train_layers=0):

    # freeze all pre-trained BERT cased layers
    if num_train_layers == 0:
      bertcased_model.trainable = False

    # partially freeze the first n pre-trained BERT cased layers
    else:
        for layer_num in range(num_train_layers):
            bertcased_model.bert.encoder.layer[layer_num].trainable = False
    
    input_ids = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}      

    bert_out = bertcased_model(bert_inputs)

    cls_token = bert_out[0][:, 0, :]

    
    hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(cls_token)

    hidden = tf.keras.layers.Dropout(dropout)(hidden)  

    f1_score = tfa.metrics.F1Score(1, threshold = 0.5)

    classification = tf.keras.layers.Dense(1, activation='sigmoid', name='classification_layer')(hidden)
    
    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])
    
    classification_model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate),
                                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), 
                                 metrics=[tf.keras.metrics.BinaryAccuracy(),
                                        tf.keras.metrics.Precision(),
                                        tf.keras.metrics.Recall(),
                                        f1_score])

    return classification_model

In [23]:
def build_bertweet_cls_model(max_sequence_length=MAX_SEQUENCE_LENGTH,
                          hidden_size=100, 
                          dropout=0.3,
                          learning_rate=0.0001,
                          num_train_layers=0):

    # freeze all pre-trained BERTweet layers
    if num_train_layers == 0:
      bertweet_model.trainable = False

    # partially freeze the first n pre-trained BERTweet layers
    else:
        for layer_num in range(num_train_layers):
            bertweet_model.roberta.encoder.layer[layer_num].trainable = False
    
    input_ids = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}      

    # Use the same bertweet model instance
    bert_out = bertweet_model(bert_inputs)

    cls_token = bert_out[0][:, 0, :]

    
    hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(cls_token)

    hidden = tf.keras.layers.Dropout(dropout)(hidden)  

    f1_score = tfa.metrics.F1Score(1, threshold = 0.5)

    classification = tf.keras.layers.Dense(1, activation='sigmoid', name='classification_layer')(hidden)
    
    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])
    
    classification_model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate),
                                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), 
                                 metrics=[tf.keras.metrics.BinaryAccuracy(),
                                        tf.keras.metrics.Precision(),
                                        tf.keras.metrics.Recall(),
                                        f1_score])

    return classification_model

## Load Models

In [24]:
# CNN
cnn_disability_model = tf.keras.models.load_model('drive/MyDrive/model_checkpoints/cnn_disability_model.keras')

In [36]:
num_train_layers=8
learning_rate=1e-6

# BERT Cased - Start with fresh bert cased instances
bertcased_disability_only_model = build_bertcased_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertcased_disability_gender_model = build_bertcased_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertcased_disability_race_model = build_bertcased_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertcased_disability_nationality_model = build_bertcased_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertcased_disability_religion_model = build_bertcased_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertcased_disability_sexual_orientation_model = build_bertcased_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertcased_gender_model = build_bertcased_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertcased_race_model = build_bertcased_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertcased_nationality_model = build_bertcased_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertcased_religion_model = build_bertcased_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertcased_sexual_orientation_model = build_bertcased_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)

# BERT Cased - Load weights
bertcased_disability_only_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERT_Cased_Disability_Only_Toxigen_Hatemoji.hdh5') # works okay
bertcased_disability_gender_model.load_weights('/content/drive/MyDrive/model_checkpoints/Official_BERT_Cased_Disability_Gender_Interwoven_disability.hdf5')
bertcased_disability_race_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERTCased_Disability_Race_Interwoven_disability.hdf5')
bertcased_disability_nationality_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERT_Cased_Disability_Nationality_Interwoven_disability.hdf5')
bertcased_disability_religion_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERT_Cased_Disability_Religion_Interwoven_disability.hdf5')
bertcased_disability_sexual_orientation_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERT_Cased_Disability_Sexual_Orientation_Interwoven_disability.hdf5')

bertcased_gender_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERT_Cased_Disability_Gender_Interwoven_gender.hdf5')
bertcased_race_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERTCased_Disability_Race_Interwoven_race.hdf5')
bertcased_nationality_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERT_Cased_Disability_Nationality_Interwoven_nationality.hdf5')
bertcased_religion_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERT_Cased_Disability_Religion_Interwoven_religion.hdf5')
bertcased_sexual_orientation_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERT_Cased_Disability_Sexual_Orientation_Interwoven_sexual_orientation.hdf5')

In [26]:
# BERTweet - Start with fresh bertweet instances
bertweet_disability_only_model = build_bertweet_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertweet_disability_gender_model = build_bertweet_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertweet_disability_race_model = build_bertweet_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertweet_disability_nationality_model = build_bertweet_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertweet_disability_religion_model = build_bertweet_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertweet_disability_sexual_orientation_model = build_bertweet_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertweet_gender_model = build_bertweet_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertweet_race_model = build_bertweet_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertweet_nationality_model = build_bertweet_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertweet_religion_model = build_bertweet_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertweet_sexual_orientation_model = build_bertweet_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)

# BERTweet - Load weights
bertweet_disability_only_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERTweet_Disability_Only_Toxigen_Hatemoji.hdh5')
bertweet_disability_gender_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERTweet_Disability_Gender_Interwoven_disability.hdf5')
bertweet_disability_race_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERTweet_Disability_Race_Interwoven_disability.hdf5')
bertweet_disability_nationality_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERTweet_Disability_Nationality_Interwoven_disability.hdf5')
bertweet_disability_religion_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERTweet_Disability_Religion_Interwoven_disability.hdf5')
bertweet_disability_sexual_orientation_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERTweet_Disability_Sexual_Orientation_Interwoven_disability.hdf5')

bertweet_gender_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERTweet_Disability_Gender_Interwoven_gender.hdf5')
bertweet_race_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERTweet_Disability_Race_Interwoven_race.hdf5')
bertweet_nationality_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERTweet_Disability_Nationality_Interwoven_nationality.hdf5')
bertweet_religion_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERTweet_Disability_Religion_Interwoven_religion.hdf5')
bertweet_sexual_orientation_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERTweet_Disability_Sexual_Orientation_Interwoven_sexual_orientation.hdf5')

# Evaluate Models

### Evaluate on ToxiGen + Hatemoji Test Set

In [27]:
# CNN --> TOXIGEN + HATEMOJI
# ==========================================
# Evaluate CNN on disability
cnn_augmented_disability_model_history = cnn_disability_model.evaluate(cnn_augmented_disability_test_input, cnn_augmented_disability_test_labels)

228/228 [==============================] - 6s 4ms/step - loss: 0.4838 - accuracy: 0.7715 - f1_score: 0.7741


In [28]:
# BERT Cased --> TOXIGEN + HATEMOJI
# ==========================================
# Evaluate BERT Cased on disability
bertcased_disability_only_model.history = bertcased_disability_only_model.evaluate(bertcased_augmented_test_inputs_disability, y_test_augmented_disability)
bertcased_disability_gender_model.history = bertcased_disability_gender_model.evaluate(bertcased_augmented_test_inputs_disability, y_test_augmented_disability)
bertcased_disability_race_model.history = bertcased_disability_race_model.evaluate(bertcased_augmented_test_inputs_disability, y_test_augmented_disability)
bertcased_disability_nationality_model.history = bertcased_disability_nationality_model.evaluate(bertcased_augmented_test_inputs_disability, y_test_augmented_disability)
bertcased_disability_religion_model.history = bertcased_disability_religion_model.evaluate(bertcased_augmented_test_inputs_disability, y_test_augmented_disability)
bertcased_disability_sexual_orientation_model.history = bertcased_disability_sexual_orientation_model.evaluate(bertcased_augmented_test_inputs_disability, y_test_augmented_disability)

# Evaluate BERT Cased on identity groups for secondary analysis
bertcased_gender_model.history = bertcased_gender_model.evaluate(bertcased_augmented_test_inputs_gender, y_test_augmented_gender)
bertcased_race_model.history = bertcased_race_model.evaluate(bertcased_augmented_test_inputs_race, y_test_augmented_race)
bertcased_nationality_model.history = bertcased_nationality_model.evaluate(bertcased_augmented_test_inputs_nationality, y_test_augmented_nationality)
bertcased_religion_model.history = bertcased_religion_model.evaluate(bertcased_augmented_test_inputs_religion, y_test_augmented_religion)
bertcased_sexual_orientation_model.history = bertcased_sexual_orientation_model.evaluate(bertcased_augmented_test_inputs_sexual_orientation, y_test_augmented_sexual_orientation)

140/140 [==============================] - 9s 41ms/step - loss: 0.3414 - binary_accuracy: 0.8528 - precision_10: 0.8259 - recall_10: 0.8812 - f1_score: 0.8527


In [29]:
# BERTweet --> TOXIGEN + HATEMOJI
# ==========================================
# Evaluate BERTweet on disability
bertweet_disability_only_model.history = bertweet_disability_only_model.evaluate(bertweet_augmented_test_inputs_disability, y_test_augmented_disability)
bertweet_disability_gender_model.history = bertweet_disability_gender_model.evaluate(bertweet_augmented_test_inputs_disability, y_test_augmented_disability)
bertweet_disability_race_model.history = bertweet_disability_race_model.evaluate(bertweet_augmented_test_inputs_disability, y_test_augmented_disability)
bertweet_disability_nationality_model.history = bertweet_disability_nationality_model.evaluate(bertweet_augmented_test_inputs_disability, y_test_augmented_disability)
bertweet_disability_religion_model.history = bertweet_disability_religion_model.evaluate(bertweet_augmented_test_inputs_disability, y_test_augmented_disability)
bertweet_disability_sexual_orientation_model.history = bertweet_disability_sexual_orientation_model.evaluate(bertweet_augmented_test_inputs_disability, y_test_augmented_disability)

# Evaluate BERTweet on identity groups for secondary analysis
bertweet_gender_model.history = bertweet_gender_model.evaluate(bertweet_augmented_test_inputs_gender, y_test_augmented_gender)
bertweet_race_model.history = bertweet_race_model.evaluate(bertweet_augmented_test_inputs_race, y_test_augmented_race)
bertweet_nationality_model.history = bertweet_nationality_model.evaluate(bertweet_augmented_test_inputs_nationality, y_test_augmented_nationality)
bertweet_religion_model.history = bertweet_religion_model.evaluate(bertweet_augmented_test_inputs_religion, y_test_augmented_religion)
bertweet_sexual_orientation_model.history = bertweet_sexual_orientation_model.evaluate(bertweet_augmented_test_inputs_sexual_orientation, y_test_augmented_sexual_orientation)

140/140 [==============================] - 10s 41ms/step - loss: 0.2975 - binary_accuracy: 0.8767 - precision_21: 0.8664 - recall_21: 0.8808 - f1_score: 0.8735


### Evaluate on HateCheck Set

In [30]:
# CNN --> HATECHECK
# ==========================================
# Evaluate CNN on disability
cnn_hatecheck_disability_model_history = cnn_disability_model.evaluate(cnn_hatecheck_disability_test_input, cnn_hatecheck_disability_test_labels)

16/16 [==============================] - 0s 10ms/step - loss: 0.5905 - accuracy: 0.7273 - f1_score: 0.8386


In [31]:
# BERT Cased --> HATE CHECK
# ==========================================
# Evaluate BERT Cased on disability
bertcased_disability_only_model = bertcased_disability_only_model.evaluate(bertcased_hatecheck_test_inputs_disability, y_test_hatecheck_disability)
bertcased_disability_gender_model = bertcased_disability_gender_model.evaluate(bertcased_hatecheck_test_inputs_disability, y_test_hatecheck_disability)
bertcased_disability_race_model = bertcased_disability_race_model.evaluate(bertcased_hatecheck_test_inputs_disability, y_test_hatecheck_disability)
bertcased_disability_nationality_model = bertcased_disability_nationality_model.evaluate(bertcased_hatecheck_test_inputs_disability, y_test_hatecheck_disability)
bertcased_disability_religion_model = bertcased_disability_religion_model.evaluate(bertcased_hatecheck_test_inputs_disability, y_test_hatecheck_disability)
bertcased_disability_sexual_orientation_model = bertcased_disability_sexual_orientation_model.evaluate(bertcased_hatecheck_test_inputs_disability, y_test_hatecheck_disability)

# Evaluate BERT Cased on identity groups for secondary analysis
bertcased_gender_model = bertcased_gender_model.evaluate(bertcased_hatecheck_test_inputs_gender, y_test_hatecheck_gender)
bertcased_race_model = bertcased_race_model.evaluate(bertcased_hatecheck_test_inputs_race, y_test_hatecheck_race)
bertcased_nationality_model = bertcased_nationality_model.evaluate(bertcased_hatecheck_test_inputs_nationality, y_test_hatecheck_nationality)
bertcased_religion_model = bertcased_religion_model.evaluate(bertcased_hatecheck_test_inputs_religion, y_test_hatecheck_religion)
bertcased_sexual_orientation_model = bertcased_sexual_orientation_model.evaluate(bertcased_hatecheck_test_inputs_sexual_orientation, y_test_hatecheck_sexual_orientation)

18/18 [==============================] - 1s 40ms/step - loss: 0.6097 - binary_accuracy: 0.7495 - precision_10: 0.7443 - recall_10: 0.9598 - f1_score: 0.8384


In [32]:
# BERTweet --> HATE CHECK
# ==========================================
# Evaluate BERTweet on disability
bertweet_disability_only_model = bertweet_disability_only_model.evaluate(bertweet_hatecheck_test_inputs_disability, y_test_hatecheck_disability)
bertweet_disability_gender_model = bertweet_disability_gender_model.evaluate(bertweet_hatecheck_test_inputs_disability, y_test_hatecheck_disability)
bertweet_disability_race_model = bertweet_disability_race_model.evaluate(bertweet_hatecheck_test_inputs_disability, y_test_hatecheck_disability)
bertweet_disability_nationality_model = bertweet_disability_nationality_model.evaluate(bertweet_hatecheck_test_inputs_disability, y_test_hatecheck_disability)
bertweet_disability_religion_model = bertweet_disability_religion_model.evaluate(bertweet_hatecheck_test_inputs_disability, y_test_hatecheck_disability)
bertweet_disability_sexual_orientation_model = bertweet_disability_sexual_orientation_model.evaluate(bertweet_hatecheck_test_inputs_disability, y_test_hatecheck_disability)

# Evaluate BERTweet on identity groups for secondary analysis
bertweet_gender_model = bertweet_gender_model.evaluate(bertweet_hatecheck_test_inputs_gender, y_test_hatecheck_gender)
bertweet_race_model = bertweet_race_model.evaluate(bertweet_hatecheck_test_inputs_race, y_test_hatecheck_race)
bertweet_nationality_model = bertweet_nationality_model.evaluate(bertweet_hatecheck_test_inputs_nationality, y_test_hatecheck_nationality)
bertweet_religion_model = bertweet_religion_model.evaluate(bertweet_hatecheck_test_inputs_religion, y_test_hatecheck_religion)
bertweet_sexual_orientation_model = bertweet_sexual_orientation_model.evaluate(bertweet_hatecheck_test_inputs_sexual_orientation, y_test_hatecheck_sexual_orientation)

18/18 [==============================] - 1s 40ms/step - loss: 0.6307 - binary_accuracy: 0.7514 - precision_21: 0.7744 - recall_21: 0.8928 - f1_score: 0.8294


### Evaluate on Jigsaw

In [33]:
# CNN --> HATECHECK
# ==========================================
# Evaluate CNN on disability
cnn_jigsaw_disability_model_history = cnn_disability_model.evaluate(cnn_jigsaw_disability_test_input, cnn_jigsaw_disability_test_labels)

49/49 [==============================] - 0s 5ms/step - loss: 0.6213 - accuracy: 0.6730 - f1_score: 0.3752


Load BERT model weights again b/c of RAM?

In [38]:
num_train_layers=8
learning_rate=1e-6

# BERT Cased - Start with fresh bert cased instances
bertcased_disability_only_model = build_bertcased_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertcased_disability_gender_model = build_bertcased_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertcased_disability_race_model = build_bertcased_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertcased_disability_nationality_model = build_bertcased_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertcased_disability_religion_model = build_bertcased_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertcased_disability_sexual_orientation_model = build_bertcased_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertcased_gender_model = build_bertcased_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertcased_race_model = build_bertcased_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertcased_nationality_model = build_bertcased_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertcased_religion_model = build_bertcased_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertcased_sexual_orientation_model = build_bertcased_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)

# BERT Cased - Load weights
bertcased_disability_only_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERT_Cased_Disability_Only_Toxigen_Hatemoji.hdh5') # works okay
bertcased_disability_gender_model.load_weights('/content/drive/MyDrive/model_checkpoints/Official_BERT_Cased_Disability_Gender_Interwoven_disability.hdf5')
bertcased_disability_race_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERTCased_Disability_Race_Interwoven_disability.hdf5')
bertcased_disability_nationality_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERT_Cased_Disability_Nationality_Interwoven_disability.hdf5')
bertcased_disability_religion_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERT_Cased_Disability_Religion_Interwoven_disability.hdf5')
bertcased_disability_sexual_orientation_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERT_Cased_Disability_Sexual_Orientation_Interwoven_disability.hdf5')

bertcased_gender_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERT_Cased_Disability_Gender_Interwoven_gender.hdf5')
bertcased_race_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERTCased_Disability_Race_Interwoven_race.hdf5')
bertcased_nationality_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERT_Cased_Disability_Nationality_Interwoven_nationality.hdf5')
bertcased_religion_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERT_Cased_Disability_Religion_Interwoven_religion.hdf5')
bertcased_sexual_orientation_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERT_Cased_Disability_Sexual_Orientation_Interwoven_sexual_orientation.hdf5')

In [39]:
# BERTweet - Start with fresh bertweet instances
bertweet_disability_only_model = build_bertweet_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertweet_disability_gender_model = build_bertweet_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertweet_disability_race_model = build_bertweet_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertweet_disability_nationality_model = build_bertweet_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertweet_disability_religion_model = build_bertweet_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertweet_disability_sexual_orientation_model = build_bertweet_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertweet_gender_model = build_bertweet_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertweet_race_model = build_bertweet_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertweet_nationality_model = build_bertweet_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertweet_religion_model = build_bertweet_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)
bertweet_sexual_orientation_model = build_bertweet_cls_model(num_train_layers=num_train_layers, learning_rate=learning_rate)

# BERTweet - Load weights
bertweet_disability_only_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERTweet_Disability_Only_Toxigen_Hatemoji.hdh5')
bertweet_disability_gender_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERTweet_Disability_Gender_Interwoven_disability.hdf5')
bertweet_disability_race_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERTweet_Disability_Race_Interwoven_disability.hdf5')
bertweet_disability_nationality_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERTweet_Disability_Nationality_Interwoven_disability.hdf5')
bertweet_disability_religion_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERTweet_Disability_Religion_Interwoven_disability.hdf5')
bertweet_disability_sexual_orientation_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERTweet_Disability_Sexual_Orientation_Interwoven_disability.hdf5')

bertweet_gender_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERTweet_Disability_Gender_Interwoven_gender.hdf5')
bertweet_race_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERTweet_Disability_Race_Interwoven_race.hdf5')
bertweet_nationality_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERTweet_Disability_Nationality_Interwoven_nationality.hdf5')
bertweet_religion_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERTweet_Disability_Religion_Interwoven_religion.hdf5')
bertweet_sexual_orientation_model.load_weights('drive/MyDrive/model_checkpoints/Official_BERTweet_Disability_Sexual_Orientation_Interwoven_sexual_orientation.hdf5')

In [40]:
# BERT Cased --> JIGSAW
# ==========================================
# Evaluate BERT Cased on disability
bertcased_disability_only_model = bertcased_disability_only_model.evaluate(bertcased_jigsaw_test_inputs_disability, y_test_jigsaw_disability)
bertcased_disability_gender_model = bertcased_disability_gender_model.evaluate(bertcased_jigsaw_test_inputs_disability, y_test_jigsaw_disability)
bertcased_disability_race_model = bertcased_disability_race_model.evaluate(bertcased_jigsaw_test_inputs_disability, y_test_jigsaw_disability)
# bertcased_disability_nationality_model = bertcased_disability_nationality_model.evaluate(bertcased_jigsaw_test_inputs_disability, y_test_jigsaw_disability) # jigsaw doesn't have nationality
bertcased_disability_religion_model = bertcased_disability_religion_model.evaluate(bertcased_jigsaw_test_inputs_disability, y_test_jigsaw_disability)
bertcased_disability_sexual_orientation_model = bertcased_disability_sexual_orientation_model.evaluate(bertcased_jigsaw_test_inputs_disability, y_test_jigsaw_disability)

# Evaluate BERT Cased on identity groups for secondary analysis
bertcased_gender_model = bertcased_gender_model.evaluate(bertcased_jigsaw_test_inputs_gender, y_test_jigsaw_gender)
bertcased_race_model = bertcased_race_model.evaluate(bertcased_jigsaw_test_inputs_race, y_test_jigsaw_race)
# bertcased_nationality_model = bertcased_nationality_model.evaluate(bertcased_jigsaw_test_inputs_nationality, y_test_jigsaw_nationality) # jigsaw doesn't have nationality
bertcased_religion_model = bertcased_religion_model.evaluate(bertcased_jigsaw_test_inputs_religion, y_test_jigsaw_religion)
bertcased_sexual_orientation_model = bertcased_sexual_orientation_model.evaluate(bertcased_jigsaw_test_inputs_sexual_orientation, y_test_jigsaw_sexual_orientation)

49/49 [==============================] - 5s 40ms/step - loss: 0.5961 - binary_accuracy: 0.7380 - precision_25: 0.2500 - recall_25: 0.2222 - f1_score: 0.2353


49/49 [==============================] - 5s 41ms/step - loss: 0.7838 - binary_accuracy: 0.5592 - precision_33: 0.3687 - recall_33: 0.7111 - f1_score: 0.4856


In [41]:
# BERTweet --> JIGSAW
# ==========================================
# Evaluate BERTweet on disability
bertweet_disability_only_model = bertweet_disability_only_model.evaluate(bertweet_jigsaw_test_inputs_disability, y_test_jigsaw_disability)
bertweet_disability_gender_model = bertweet_disability_gender_model.evaluate(bertweet_jigsaw_test_inputs_disability, y_test_jigsaw_disability)
bertweet_disability_race_model = bertweet_disability_race_model.evaluate(bertweet_jigsaw_test_inputs_disability, y_test_jigsaw_disability)
# bertweet_disability_nationality_model = bertweet_disability_nationality_model.evaluate(bertweet_jigsaw_test_inputs_disability, y_test_jigsaw_disability) # jigsaw doesn't have nationality
bertweet_disability_religion_model = bertweet_disability_religion_model.evaluate(bertweet_jigsaw_test_inputs_disability, y_test_jigsaw_disability)
bertweet_disability_sexual_orientation_model = bertweet_disability_sexual_orientation_model.evaluate(bertweet_jigsaw_test_inputs_disability, y_test_jigsaw_disability)

# Evaluate BERTweet on identity groups for secondary analysis
bertweet_gender_model = bertweet_gender_model.evaluate(bertweet_jigsaw_test_inputs_gender, y_test_jigsaw_gender)
bertweet_race_model = bertweet_race_model.evaluate(bertweet_jigsaw_test_inputs_race, y_test_jigsaw_race)
# bertweet_nationality_model = bertweet_nationality_model.evaluate(bertweet_jigsaw_test_inputs_nationality, y_test_jigsaw_nationality) # jigsaw doesn't have nationality
bertweet_religion_model = bertweet_religion_model.evaluate(bertweet_jigsaw_test_inputs_religion, y_test_jigsaw_religion)
bertweet_sexual_orientation_model = bertweet_sexual_orientation_model.evaluate(bertweet_jigsaw_test_inputs_sexual_orientation, y_test_jigsaw_sexual_orientation)

49/49 [==============================] - 5s 40ms/step - loss: 0.6099 - binary_accuracy: 0.7776 - precision_34: 0.3575 - recall_34: 0.2832 - f1_score: 0.3160


49/49 [==============================] - 6s 40ms/step - loss: 0.9778 - binary_accuracy: 0.5260 - precision_44: 0.3609 - recall_44: 0.8044 - f1_score: 0.4983


# Analysis
Resources:
- AllenNLP
  - Official github: https://github.com/allenai/allennlp
  - Example results: https://santiagof.medium.com/model-interpretability-making-your-model-confesses-saliency-maps-5b14c1e04f9e
  - Official demo: https://demo.allennlp.org/sentiment-analysis/roberta-sentiment-analysis
